In [ ]:
import face_recognition
import cv2
import numpy as np
import os
import requests
from datetime import datetime

# Path to the folder containing person images
path = 'persons'
# Google Form URL for submitting data
google_form_url = 'https://docs.google.com/forms/d/107bGqU_hECzNG_QGzWj6MqVGTwXE5A8djuwv4-tWuYg/edit'

# Load images and person names
images = []
classNames = []
personsList = os.listdir(path)

# Loop through the list of people and load their images and names
for cl in personsList:
    curPerson = cv2.imread(f'{path}/{cl}')
    if curPerson is not None:
        images.append(curPerson)
        classNames.append(os.path.splitext(cl)[0].upper())
    else:
        print(f"Warning: Could not load image for {cl}")

# Function to find face encodings
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(img)
        if len(encodings) > 0:
            encodeList.append(encodings[0])
        else:
            print("Warning: No face found in one of the images.")
    return encodeList

# Get encodings for known faces
encodeListKnown = findEncodings(images)
if len(encodeListKnown) == 0:
    print("No faces were encoded. Exiting.")
    exit()  # Exit if no encodings were found
else:
    print('Encoding Complete.')

# Function to submit data to Google Form
def submit_to_google_form(name, attendance_status, school_id):
    form_data = {
        'entry.616242507': name,  # Field ID for the name in Google Form
        'entry.232200178': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),  # Field ID for date and time
        'entry.1792202557': attendance_status,  # Field ID for attendance status (Present/Absent)
        'entry.481307232': school_id  # Field ID for School ID
    }

    try:
        # Send data to Google Form with a timeout to avoid long waits
        response = requests.post(google_form_url, data=form_data, timeout=10)

        # Check if submission is successful
        if response.status_code == 200:
            print(f"Data successfully submitted for {name} to Google Sheets!")
        else:
            print(f"Error occurred while submitting data for {name}. Status Code: {response.status_code}, Response: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to submit data for {name}. Error: {str(e)}")

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, img = cap.read()

    if ret:
        # Resize the frame for faster processing and convert it to RGB
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        # Detect faces in the current frame and compute their encodings
        faceCurrentFrame = face_recognition.face_locations(imgS)
        encodeCurrentFrame = face_recognition.face_encodings(imgS, faceCurrentFrame)

        for encodeFace, faceLoc in zip(encodeCurrentFrame, faceCurrentFrame):
            # Compare the current face with known faces and get the best match
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex]
                print(name)

                # Submit attendance data to Google Form
                submit_to_google_form(name, 'Present', 'School123')  # Example school_id

                # Scale the face location back to original size for drawing
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

                # Draw a rectangle around the face and display the name
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

        # Display the processed video feed
        cv2.imshow('Face Recognition', img)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print("Error: Failed to capture image from webcam.")
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

